In [2]:
import pandas as pd
import json
import re
from konlpy.tag import Okt
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.stats import pearsonr
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [3]:
with open("./Documents/c_places.json", "r", encoding="utf8") as f:
    contents = f.read() # string 타입
    json_data = json.loads(contents)

with open("./Documents/delete_list.txt", encoding='cp949') as f:
    delete_list = f.readlines()

In [4]:
place_list = []
for i in json_data["place"]:
    place_list.append(i)

In [5]:
place_reviews = dict.fromkeys(place_list)

In [6]:
for i in place_list:
    reviews = []
    if 'review' in json_data["place"][i]:
        length = len(json_data["place"][i]["review"])
        for j in range(0, length):
            review = json_data["place"][i]["review"][j][0]
            reviews.append(review)
            corpus = " ".join(reviews)
    place_reviews[i] = corpus

In [7]:
place_reviews

{'526': '치킨난반은 쏘쏘였는데 모시바지락은👍\n전통주가 많아서 좋았음\n3차나 간술하기 굿!!!\n테이블이 적어서 아쉽 항상 사람이 많아서 가보고싶었던 곳!!!\n\n결과는 대만족 입니다ㅠㅠㅠ \n안주 모두 다 너무 맛있어요ㅠㅠㅠㅠ\n사장님도 정말 친절하시고 친구 모임이나\n커플끼리 방문하시면 좋을 것 같아요!!!\n\n대 추천👍🏻👍🏻 늘 웨이팅이 있는 공릉동 핫플입니다 두번째 방문이었는데 두반장크림가라아게 진짜 미쳤어요... 새로운 안주를 시킬때마다 너무너무너무 맛있습니다 !! 직원분도 너무 친절하시고 분위기가 좋아요 이제부터 여기 단골 예약!\n마라전골은 저렇게 우삼겹이랑 야채도 더 주세요\n양도 많고 음식이 다 맛있어요\n덕분에 과음했읍니다',
 '5412': '인테리어가 특별한 것 같지는 않은데 특별해 보이는 매력적인 곳이에요! 음식들도 익숙한 맛이라고 해도 낯선 느낌으로 예쁘게 나와서 넘 좋답니다 +_+ 가격대는 좀 있어도 또 가고 싶어요! 역시 너무 맛있네요 ☺️♥️ 유명한 메뉴들만 시켜서 먹었습니다 만족스러워요 ㅎㅎ 주류주문이 필수라 어쩔 수 없이 대낮부터 술을 먹었지만 기분 좋았네요~! 여자 둘이서 가서 메뉴 세개 거뜬하네요 (둘이 워낙 잘먹어서요 ㅋ ) 여튼 담에 멤버 하나둘 더 추가해서 가봐야겠어요💕 예약하고 방문하는 걸 추천, 워크인도 받기는 하나 못 앉을 가능성이 높다. 양이 적고 주류 주문 필수,, 육회김밥은 쏘쏘 미나리새우전이랑 감태멍게초밥인가?는 맛있었다. 재방문의사는 있는데 한 내년쯤? 올려나.. 정말 맛있고 친절하세요!\n서빙해주실 때도 하나하나 설명해주시면서 음식 주셔서 감사했어요. \n특히 미나리 새우전이 정말정말 맛있었어요\n다만 아쉬운 건 가격이 쎄요 음식가격은 그만큼 맛있으니 그렇다고 하지만 술 가격이 복순도가 막걸리가 한병에 29000원이에요 ㅠ 저렴한 가격의 막걸리는 아니지만 만이천원짜리인데ㅠ ㅠ\n가격이 아쉬웠지만 정말 맛있게 잘 먹었어요!!!',
 '&meal': '예쁘고요. 전반적으로 조화롭고 맛있어요.\

In [8]:
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()

In [9]:
add_stopwords = delete_list[0].split()
for word in add_stopwords:
    stopwords.append(word)

In [10]:
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 정규 표현식 처리
    result = hangul.sub('', text)
    okt = Okt()  # 형태소 추출
    nouns = okt.nouns(result)
    nouns = [x for x in nouns if len(x) > 1]  # 한글자 키워드 제거
    nouns = [x for x in nouns if x not in stopwords]  # 불용어 제거
    return nouns

In [11]:
words = []
for i in place_list:
    tc = text_cleaning(place_reviews[i])
    words.append(tc)
    place_reviews[i] = tc
    
sum_words = sum(words, [])
#print(sum_words)

OpenJDK 64-Bit Server VM warning: Attempt to protect stack guard pages failed.
OpenJDK 64-Bit Server VM warning: Attempt to deallocate stack guard pages failed.


In [26]:
for i in place_list:
    try:
        count = Counter(place_reviews[i]).most_common(n=10)
        tmp = []
        for j in range(len(count)):
            tmp.append(count[j][0])
        
        json_data['place'][i]['keyword'] = tmp
    except:
        print(i + ' has no review')

In [29]:
json_data['place']['PKM가든 레스토랑&카페']

{'category': 'restaurant',
 'image': ['https://search.pstatic.net/common/?autoRotate=true&type=w278_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20180704_122%2F1530663704600O65p7_JPEG%2FkGEM0H9TqPALyK9USc6_qFPB.jpg',
  'https://search.pstatic.net/common/?autoRotate=true&type=w278_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20180704_80%2F1530663711221RTMWJ_JPEG%2FipV__co1YLIH1HC4RAYLGb2w.jpg',
  'https://search.pstatic.net/common/?autoRotate=true&type=w278_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20180704_247%2F1530663714958u088N_JPEG%2FWbVoZ3cinYR--JdIv1iIrdA1.jpg'],
 'link': 'https://www.instagram.com/pkmgarden/',
 'map': '서울 종로구 삼청로7길 40 피케이엠갤러리',
 'review': [['시그니처로만 먹엇는데 너무 맛있어요! 페스츄리피자도 정말 맛있는데 알리오 올리오도.. 쵝오!!\n주차 2시간이라서 조금일찍도착하고 2시간 안에 나갈라고 했는데 이전팀때문에 여의치 않았어요. 참고하세요~',
   'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=w750&src=https%3A%2F%2Fnaverbooking-phinf.pstatic.net%2F20220720_279%2F1658288834044NM5Ud_JPEG%2Fimage.jpg',
   'https://s

In [28]:
with open("k_places.json", 'w') as outfile:
    json.dump(json_data, outfile)

In [11]:
vect = CountVectorizer(tokenizer = lambda x: text_cleaning(x))
bow_vect = vect.fit_transform(place_reviews)
word_list = vect.get_feature_names()
count_list = bow_vect.toarray().sum(axis=0)

In [12]:
tfidf_vectorizer = TfidfTransformer()
tf_idf_vect = tfidf_vectorizer.fit_transform(bow_vect)

In [13]:
countvect_df = pd.DataFrame(tf_idf_vect.toarray(), columns = sorted(vect.vocabulary_))
countvect_df.index = place_reviews.keys()
countvect_df

,가든,가락,가람,가로수길,가마,가마솥,가배,가부,가산,가산디지털단지역,...,휴식,흑석역,흥남,희릿,히노,히든,히어로,히카리,히트,힛더
526,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
5412,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
&meal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
101번지남산돈까스 현대시티아울렛가산점,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.317397,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
1954무한등심 잠실점,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
히든브라운,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.720728,0.0,0.000000,0.0,0.000000
히카리 우동,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.776823,0.0,0.000000
히피스베이글,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
힐사이드테이블,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000


In [14]:
a = countvect_df.corr(method='pearson')

In [15]:
vect = CountVectorizer() # Counter Vectorizer 객체 생성

# 문장을 Counter Vectorizer 형태로 변형 
countvect = vect.fit_transform(place_reviews) 
countvect.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [16]:
countvect_df = pd.DataFrame(countvect.toarray(), columns = sorted(vect.vocabulary_))
countvect_df.index = place_reviews

In [17]:
a = countvect_df.corr(method='pearson')

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
tfvect = vect.fit(place_reviews)

In [19]:
tfidv_df = pd.DataFrame(tfvect.transform(place_reviews).toarray(), columns = sorted(vect.vocabulary_))
tfidv_df.index = place_reviews

In [34]:
b = tfidv_df.corr(method='pearson')

In [21]:
# movie title와 id를 매핑할 dictionary를 생성해줍니다. 
place2id = {}
for i, c in enumerate(place_reviews): place2id[i] = c

# id와 movie title를 매핑할 dictionary를 생성해줍니다. 
id2place = {}
for i, c in place2id.items(): id2place[c] = i

In [42]:
#idx = place2id[14]
sim_scores = [(i, c) for i, c in enumerate(b['힐사이드테이블']) if i != '힐사이드테이블'] # 자기 자신을 제외한 영화들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:10] # 상위 10개의 인덱스와 유사도를 추출 

[(2392, 1.0),
 (268, -0.00048332527791203204),
 (190, -0.0004833252779120322),
 (178, -0.00048332527791203225),
 (427, -0.00048332527791203225),
 (115, -0.0004833252779120323),
 (386, -0.0004833252779120323),
 (452, -0.0004833252779120323),
 (635, -0.0004833252779120323),
 (584, -0.0004833252779120326)]

In [46]:
place2id[268]

'노량해전 노량진횟집'

In [47]:
place2id[268]

list